In [1]:
import torch
import matplotlib.pyplot as plt
import supersuit as ss
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.vec_env import VecFrameStack
from ctf_env import CaptureTheFlagPZ

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Initialize PettingZoo Env
env = CaptureTheFlagPZ(render_mode="rgb_array")

# Convert to SB3 Vector Env
vec_env = ss.pettingzoo_env_to_vec_env_v1(env)
vec_env = ss.concat_vec_envs_v1(vec_env, num_vec_envs=4, num_cpus=0, base_class='stable_baselines3')

print(f"Observation Space: {vec_env.observation_space.shape}")

# --- FIX 1: SAFETY NET (CHECKPOINTS) ---
# This saves the model every 100,000 steps. 
# If the kernel crashes at step 900k, you only lose the last bit, not everything.
checkpoint_callback = CheckpointCallback(
    save_freq=100_000,
    save_path="./logs/",
    name_prefix="ctf_model"
)

# --- FIX 2: MEMORY OPTIMIZATION ---
model = PPO(
    "MultiInputPolicy", 
    vec_env, 
    verbose=1, 
    # REDUCED from 4096. 512 is much safer for a GTX 1080.
    # This does NOT affect intelligence, only training speed (might actually be faster due to less swapping).
    batch_size=512,  
    learning_rate=1e-4, 
    ent_coef=0.03,    # Lowered slightly as training progresses
    # REDUCED from 2048. Shorter rollouts mean less RAM usage.
    n_steps=1024,     
    device=device
)

print("Starting Training...")

# Pass the callback here
model.learn(total_timesteps=1_000_000, callback=checkpoint_callback)

print("Training Finished!")

# Save Final Model
model.save("ctf_champion_1m_offensive")

DEBUG: blue_2 PICKED UP FLAG
DEBUG: red_2 PICKED UP FLAG
DEBUG: blue_1 PICKED UP FLAG
------------------------------
| time/              |       |
|    fps             | 675   |
|    iterations      | 1     |
|    time_elapsed    | 24    |
|    total_timesteps | 16384 |
------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 559          |
|    iterations           | 2            |
|    time_elapsed         | 58           |
|    total_timesteps      | 32768        |
| train/                  |              |
|    approx_kl            | 0.0068273772 |
|    clip_fraction        | 0.00014      |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.1         |
|    explained_variance   | -0.0098      |
|    learning_rate        | 0.0001       |
|    loss                 | 2.66         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.000748    |
|    valu

In [2]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA version:", torch.version.cuda)
    print("GPU name:", torch.cuda.get_device_name(torch.cuda.current_device()))

PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
GPU name: NVIDIA GeForce GTX 1080


In [3]:
# --- NY CELL: FORTSÄTT TRÄNA (RESUME) ---

# 1. Definiera hur mycket MER vi ska träna
extra_timesteps = 2_000_000 
load_filename = "ctf_champion_1m_offensive"      # Filen från förra cellen
save_filename = "ctf_champion_3m_offensive"   # Det nya namnet

print(f"Resuming training from {load_filename}...")

# 2. Ladda modellen och koppla den till din existerande vec_env
# Vi behåller samma device och miljöinställningar
try:
    model = PPO.load(load_filename, env=vec_env, device=device)
    
    # Valfritt: Om du vill att de ska utforska lite mindre nu när de kan grunderna,
    # kan du sänka ent_coef här (t.ex. till 0.05), annars behålls 0.1 från originalet.
    # model.ent_coef = 0.05 

    print("Model loaded successfully. Starting training...")
    
    # 3. Träna vidare
    # reset_num_timesteps=False är nyckeln för att fortsätta timeregnskapet
    model.learn(total_timesteps=extra_timesteps, reset_num_timesteps=False)
    
    print("Extra training finished!")
    
    # 4. Spara den uppdaterade versionen
    model.save(save_filename)
    print(f"Model saved as: {save_filename}")

except FileNotFoundError:
    print(f"FEL: Hittade inte filen '{load_filename}.zip'. Kontrollera namnet!")

Resuming training from ctf_champion_1m_offensive...
Model loaded successfully. Starting training...
DEBUG: red_1 PICKED UP FLAG
DEBUG: red_2 PICKED UP FLAG
DEBUG: red_1 PICKED UP FLAG
--------------------------------
| time/              |         |
|    fps             | 880     |
|    iterations      | 1       |
|    time_elapsed    | 18      |
|    total_timesteps | 1032192 |
--------------------------------
DEBUG: red_2 PICKED UP FLAG
DEBUG: blue_2 PICKED UP FLAG
DEBUG: red_2 PICKED UP FLAG
DEBUG: blue_2 PICKED UP FLAG
DEBUG: blue_1 PICKED UP FLAG
DEBUG: red_1 PICKED UP FLAG
DEBUG: red_1 PICKED UP FLAG
DEBUG: red_2 PICKED UP FLAG
-----------------------------------------
| time/                   |             |
|    fps                  | 530         |
|    iterations           | 2           |
|    time_elapsed         | 61          |
|    total_timesteps      | 1048576     |
| train/                  |             |
|    approx_kl            | 0.011270661 |
|    clip_fraction    

In [4]:
# 1. Ladda din 3M-checkpoint
model = PPO.load("ctf_champion_3m_offensive", env=vec_env, device=device)

# 2. Ge dem lite mer utforskning igen (0.03 istället för 0.01)
# Och håll kvar LR på 1e-4 en stund till så de faktiskt lär sig av sina scores
model.ent_coef = 0.03
model.learning_rate = 1e-4

print("Resuming with balanced exploration to prevent 'freezing'...")
model.learn(total_timesteps=2_000_000, reset_num_timesteps=False)
model.save("alex")

Resuming with balanced exploration to prevent 'freezing'...
DEBUG: red_1 PICKED UP FLAG
DEBUG: red_2 PICKED UP FLAG
DEBUG: red_1 PICKED UP FLAG
DEBUG: blue_1 PICKED UP FLAG
DEBUG: red_1 PICKED UP FLAG
DEBUG: red_2 PICKED UP FLAG
DEBUG: red_1 PICKED UP FLAG
DEBUG: red_1 PICKED UP FLAG
DEBUG: red_2 PICKED UP FLAG
--------------------------------
| time/              |         |
|    fps             | 712     |
|    iterations      | 1       |
|    time_elapsed    | 23      |
|    total_timesteps | 3047424 |
--------------------------------
DEBUG: red_1 PICKED UP FLAG
DEBUG: red_1 PICKED UP FLAG
DEBUG: red_1 PICKED UP FLAG
DEBUG: red_2 PICKED UP FLAG
DEBUG: red_2 PICKED UP FLAG
DEBUG: red_1 PICKED UP FLAG
DEBUG: red_2 PICKED UP FLAG
-----------------------------------------
| time/                   |             |
|    fps                  | 507         |
|    iterations           | 2           |
|    time_elapsed         | 64          |
|    total_timesteps      | 3063808     |
| train/